<a href="https://colab.research.google.com/github/BetaPrimePsi/BetaPrimePsi/blob/main/CS686_Assignment_4_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##### License { display-mode: "form" }
# Copyright 2019 DeepMind Technologies Ltd. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# OpenSpiel: RRPS Example

* This Colab gets you started with installing OpenSpiel and its dependencies.
* OpenSpiel is a framework for reinforcement learning in games.
* For a longer intro to OpenSpiel, see [the tutorial video](https://www.youtube.com/watch?v=8NCPqtPwlFQ), [documentation](https://openspiel.readthedocs.io/en/latest/), or [API reference](https://openspiel.readthedocs.io/en/latest/api_reference.html).
* This colab also includes examples of how to get started with the Roshambo/RRPS environment and bots. It is based on [roshambo_population_example.py](https://github.com/google-deepmind/open_spiel/blob/master/open_spiel/python/examples/roshambo_population_example.py)
* For more info on Roshambo, see the [RRPS benchmark paper](https://openreview.net/pdf?id=gQnJ7ODIAx).

## Install

Install OpenSpiel via pip:

Note that if you are not using a colab, then you would use
python3 -m pip install open_spiel
Additional information about installing OpenSpiel can be found at the links above.


In [ ]:
!pip install --upgrade open_spiel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 2.6 MB/s eta 0:00:00


# Simple example of OpenSpiel API: uniform random trajectory on Tic-Tac-Toe

This example is not used for RRPS, but it shows how to load a game in OpenSpiel, how to access a game state, and implements a random strategy.

In [ ]:
import numpy as np
import pyspiel

game = pyspiel.load_game("tic_tac_toe")
state = game.new_initial_state()

while not state.is_terminal():
  state.apply_action(np.random.choice(state.legal_actions()))
  print(str(state) + '\n')

...
...
x..

.o.
...
x..

.o.
.x.
x..

.o.
.x.
xo.

.o.
xx.
xo.

oo.
xx.
xo.

oo.
xx.
xox

oo.
xxo
xox

oox
xxo
xox



# Getting started with RRPS


In [ ]:
# Imports
import numpy as np

from open_spiel.python import rl_agent
from open_spiel.python import rl_environment
import pyspiel


In [ ]:
# Some helper classes and functions.
# DO NOT CHANGE.

class BotAgent(rl_agent.AbstractAgent):
  """Agent class that wraps a bot.

  Note, the environment must include the OpenSpiel state in its observations,
  which means it must have been created with use_full_state=True.

  This is a simple wrapper that lets the RPS bots be interpreted as agents under
  the RL API.
  """

  def __init__(self, num_actions, bot, name="bot_agent"):
    assert num_actions > 0
    self._bot = bot
    self._num_actions = num_actions

  def restart(self):
    self._bot.restart()

  def step(self, time_step, is_evaluation=False):
    # If it is the end of the episode, don't select an action.
    if time_step.last():
      return
    _, state = pyspiel.deserialize_game_and_state(
        time_step.observations["serialized_state"])
    action = self._bot.step(state)
    probs = np.zeros(self._num_actions)
    probs[action] = 1.0
    return rl_agent.StepOutput(action=action, probs=probs)


#  We will use this function to evaluate the agents. Do not change.

def eval_agents(env, agents, num_players, num_episodes, verbose=False):
  """Evaluate the agent.

  Runs a number of episodes and returns the average returns for each agent as
  a numpy array.

  Arguments:
    env: the RL environment,
    agents: a list of agents (size 2),
    num_players: number of players in the game (for RRPS, this is 2),
    num_episodes: number of evaluation episodes to run.
    verbose: whether to print updates after each episode.
  """
  sum_episode_rewards = np.zeros(num_players)
  for ep in range(num_episodes):
    for agent in agents:
      # Bots need to be restarted at the start of the episode.
      if hasattr(agent, "restart"):
        agent.restart()
    time_step = env.reset()
    episode_rewards = np.zeros(num_players)
    while not time_step.last():
      agents_output = [
          agent.step(time_step, is_evaluation=True) for agent in agents
      ]
      action_list = [agent_output.action for agent_output in agents_output]
      time_step = env.step(action_list)
      episode_rewards += time_step.rewards
    sum_episode_rewards += episode_rewards
    if verbose:
      print(f"Finished episode {ep}, "
            + f"avg returns: {sum_episode_rewards / (ep+1)}")

  return sum_episode_rewards / num_episodes


def print_roshambo_bot_names_and_ids(roshambo_bot_names):
  print("Roshambo bot population:")
  for i in range(len(roshambo_bot_names)):
    print(f"{i}: {roshambo_bot_names[i]}")

def create_roshambo_bot_agent(player_id, num_actions, bot_names, pop_id):
  name = bot_names[pop_id]
  # Creates an OpenSpiel bot with the default number of throws
  # (pyspiel.ROSHAMBO_NUM_THROWS). To create one for a different number of
  # throws per episode, add the number as the third argument here.
  bot = pyspiel.make_roshambo_bot(player_id, name)
  return BotAgent(num_actions, bot, name=name)


#The following functions are used to load the bots from the original RRPS competition.

In [ ]:
# Some basic info and initialize the population

# print(pyspiel.ROSHAMBO_NUM_BOTS)    # 43 bots
# print(pyspiel.ROSHAMBO_NUM_THROWS)  # 1000 steps per episode

# The recall is how many of the most recent actions are presented to the RL
# agents as part of their observations. Note: this is just for the RL agents
# like DQN etc... every bot has access to the full history.
RECALL = 20

# The population of 43 bots. See the RRPS paper for high-level descriptions of
# what each bot does.

print("Loading bot population...")
pop_size = pyspiel.ROSHAMBO_NUM_BOTS
print(f"Population size: {pop_size}")
roshambo_bot_names = pyspiel.roshambo_bot_names()
roshambo_bot_names.sort()
print_roshambo_bot_names_and_ids(roshambo_bot_names)

bot_id = 0
roshambo_bot_ids = {}
for name in roshambo_bot_names:
  roshambo_bot_ids[name] = bot_id
  bot_id += 1


Loading bot population...
Population size: 43
Roshambo bot population:
0: actr_lag2_decay
1: adddriftbot2
2: addshiftbot3
3: antiflatbot
4: antirotnbot
5: biopic
6: boom
7: copybot
8: debruijn81
9: driftbot
10: flatbot3
11: foxtrotbot
12: freqbot2
13: granite
14: greenberg
15: halbot
16: inocencio
17: iocainebot
18: marble
19: markov5
20: markovbails
21: mixed_strategy
22: mod1bot
23: multibot
24: peterbot
25: phasenbott
26: pibot
27: piedra
28: predbot
29: r226bot
30: randbot
31: robertot
32: rockbot
33: rotatebot
34: russrocker4
35: shofar
36: sunCrazybot
37: sunNervebot
38: sweetrock
39: switchalot
40: switchbot
41: textbot
42: zq_move


#Example showing how to load to agents from the RRPS bot population and evalute them against each other.

In [ ]:
# Example: create an RL environment, and two agents from the bot population and
# evaluate these two agents head-to-head.

# Note that the include_full_state variable has to be enabled because the
# BotAgent needs access to the full state.
env = rl_environment.Environment(
    "repeated_game(stage_game=matrix_rps(),num_repetitions=" +
    f"{pyspiel.ROSHAMBO_NUM_THROWS}," +
    f"recall={RECALL})",
    include_full_state=True)
num_players = 2
num_actions = env.action_spec()["num_actions"]
# Learning agents might need this:
info_state_size = env.observation_spec()["info_state"][0]

# Create two bot agents
p0_pop_id = 0   # actr_lag2_decay
p1_pop_id = 1   # adddriftbot2
agents = [
    create_roshambo_bot_agent(0, num_actions, roshambo_bot_names, p0_pop_id),
    create_roshambo_bot_agent(1, num_actions, roshambo_bot_names, p1_pop_id)
]

print("Starting eval run.")
avg_eval_returns = eval_agents(env, agents, num_players, 10, verbose=True)

print("Avg return ", avg_eval_returns)

Starting eval run.
Finished episode 0, avg returns: [ 39. -39.]
Finished episode 1, avg returns: [ 22. -22.]
Finished episode 2, avg returns: [ 20.33333333 -20.33333333]
Finished episode 3, avg returns: [ 18. -18.]
Finished episode 4, avg returns: [ 11.6 -11.6]
Finished episode 5, avg returns: [ 12.5 -12.5]
Finished episode 6, avg returns: [ 20.42857143 -20.42857143]
Finished episode 7, avg returns: [ 15.875 -15.875]
Finished episode 8, avg returns: [ 12.33333333 -12.33333333]
Finished episode 9, avg returns: [ 12.1 -12.1]
Avg return  [ 12.1 -12.1]


# Data Collection

In [ ]:
import torch
import numpy as np
from tqdm import tqdm

def collect_rrps_data(env, bot_names, episodes_per_pair=2):
    trajectories = []
    pop_size = len(bot_names)

    for i in range(pop_size):
        for j in range(pop_size):
            # Create agents for both bots
            ag0 = create_roshambo_bot_agent(0, num_actions, bot_names, i)
            ag1 = create_roshambo_bot_agent(1, num_actions, bot_names, j)
            agents = [ag0, ag1]

            for ep in range(episodes_per_pair):
                for ag in agents:
                    if hasattr(ag, "restart"):
                        ag.restart()

                ts = env.reset()
                ep_hist = []

                while not ts.last():
                    outputs = [ag.step(ts) for ag in agents]
                    acts = [out.action for out in outputs]
                    ep_hist.append(tuple(acts))
                    ts = env.step(acts)

                trajectories.append(ep_hist)

    return trajectories



In [ ]:
def collect_pairs_data(env, bot_names, pairs, episodes_per_pair=1):
    trajectories = []

    for (i, j) in pairs:

        # Agent 0
        if i == 43:
            ag0 = MyAgent(num_actions)
        else:
            ag0 = create_roshambo_bot_agent(0, num_actions, bot_names, i)

        # Agent 1
        if j == 43:
            ag1 = MyAgent(num_actions)
        else:
            ag1 = create_roshambo_bot_agent(1, num_actions, bot_names, j)

        agents = [ag0, ag1]

        for ep in range(episodes_per_pair):
            for ag in agents:
                if hasattr(ag, "restart"):
                    ag.restart()

            ts = env.reset()
            ep_hist = []

            while not ts.last():
                outputs = [ag.step(ts) for ag in agents]
                acts = [out.action for out in outputs]
                ep_hist.append(tuple(acts))
                ts = env.step(acts)

            trajectories.append(ep_hist)

    return trajectories


# Turn Trajectories into Supervised Samples

In [ ]:
PAD = 9   # special pad token

def make_training_samples(trajectories, window=100):
    X = []
    Y = []

    for traj in trajectories:
        for t in range(1, len(traj)):
            past = traj[max(0, t-window):t]

            tokens = [(a0*3 + a1) for (a0, a1) in past]

            if len(tokens) < window:
                tokens = [PAD] * (window - len(tokens)) + tokens
            else:
                tokens = tokens[-window:]

            target_a0, target_a1 = traj[t]
            target = target_a0*3 + target_a1

            X.append(tokens)
            Y.append(target)

    return torch.tensor(X), torch.tensor(Y)


In [ ]:
def collect_rrps_windows_from_pairs(pairs,              # list of (id0, id1)
                                    env=env,
                                    bot_names=roshambo_bot_names,
                                    num_actions=num_actions,
                                    episodes_per_pair=1,
                                    window=100):
    PAD = 9
    def encode_joint(a0, a1): return 3*a0 + a1

    X_list, Y_list = [], []
    id0_list, id1_list = [], []

    for (i, j) in pairs:

        # agent 0
        if i == 43:
            ag0 = MyAgent(num_actions, name="trans_agent")
        else:
            ag0 = create_roshambo_bot_agent(0, num_actions, bot_names, i)

        # agent 1
        if j == 43:
            ag1 = MyAgent(num_actions, name="trans_agent")
        else:
            ag1 = create_roshambo_bot_agent(1, num_actions, bot_names, j)

        agents = [ag0, ag1]

        for ep in range(episodes_per_pair):
            for ag in agents:
                if hasattr(ag, "restart"):
                    ag.restart()

            ts = env.reset()
            ep_hist = []

            while not ts.last():
                outputs = [ag.step(ts) for ag in agents]
                acts = [out.action for out in outputs]
                ep_hist.append(tuple(acts))
                ts = env.step(acts)

            # convert episode to token sequence
            tokens = [encode_joint(a0, a1) for (a0, a1) in ep_hist]

            # sliding windows
            for t in range(1, len(tokens)):
                start = max(0, t - window)
                seq = tokens[start:t]

                if len(seq) < window:
                    seq = [PAD] * (window - len(seq)) + seq

                X_list.append(seq)
                Y_list.append(tokens[t])
                id0_list.append(i)
                id1_list.append(j)

    X = torch.tensor(X_list, dtype=torch.long)
    Y = torch.tensor(Y_list, dtype=torch.long)
    id0 = torch.tensor(id0_list, dtype=torch.long)
    id1 = torch.tensor(id1_list, dtype=torch.long)

    return X, Y, id0, id1


In [ ]:
def collect_rrps_windows_two_lists(listA,
                                   listB,
                                   env=env,
                                   bot_names=roshambo_bot_names,
                                   num_actions=num_actions,
                                   episodes_per_pair=1,
                                   window=100):
    pairs = [(i, j) for i in listA for j in listB]
    return collect_rrps_windows_from_pairs(pairs,
        episodes_per_pair=episodes_per_pair,
        window=window
    )


In [ ]:
pairs = [
(0, 30),   # actr_lag2_decay vs randbot
(1, 30),   # adddriftbot2 vs randbot
(3, 30),   # antiflatbot vs randbot
(4, 33),   # antirotnbot vs rotatebot
(5, 30),   # biopic vs randbot
(6, 30),   # boom vs randbot
(7, 30),   # copybot vs randbot
(8, 30),   # debruijn81 vs randbot
(9, 30),   # driftbot vs randbot
(10, 32),  # flatbot3 vs rockbot
(11, 30),  # foxtrotbot vs randbot
(13, 30),  # granite vs randbot
(14, 30),  # greenberg vs randbot
(15, 30),  # halbot vs randbot
(16, 30),  # inocencio vs randbot
(18, 30),  # marble vs randbot
(20, 30),  # markovbails vs randbot
(21, 32),  # mixed_strategy vs rockbot
(23, 30),  # multibot vs randbot
(24, 30),  # peterbot vs randbot
(26, 30),  # pibot vs randbot
(27, 32),  # piedra vs rockbot
(31, 32),  # robertot vs rockbot
(34, 30),  # russrocker4 vs randbot
(35, 30),  # shofar vs randbot
(38, 32),  # sweetrock vs rockbot
(40, 30),  # switchbot vs randbot
(41, 30),  # textbot vs randbot
(42, 30),  # zq_move vs randbot

(12, 30),  # vs randbot
(12, 32),  # vs rockbot

(12, 30),  # vs randbot
(12, 32),  # vs rockbot

(17, 30),  # vs randbot
(17, 33),  # vs rotatebot

(19, 30),  # vs randbot
(19, 33),  # vs rotatebot

(25, 33),  # vs rotatebot (reveals phase)
(25, 30),  # vs randbot (tests stability)

(22, 33),  # vs rotatebot (mod pattern emerges)
(22, 12),  # vs freqbot2 (long-range pattern exposure)

(28, 30),  # vs randbot
(28, 12),  # vs freqbot2 (tests prediction ability)

(29, 30),  # baseline
(29, 33),  # high structure probe

(36, 30),  # vs randbot
(36, 33),  # vs rotatebot
(36, 17),  # vs iocainebot (meta-chaos interaction)

(37, 30),  # vs randbot
(37, 12),  # vs freqbot2
(37, 33),  # vs rotatebot

(39, 30),  # vs randbot
(39, 33),  # vs rotatebot
(39, 12),  # vs freqbot2

(32, 33),  # rockbot vs rotatebot — contrast of static vs periodic
(12, 19),  # freqbot2 vs markov5 — predictor vs memory-based
(17, 19)  # iocainebot vs markov5 — meta-strategy vs long-horizon
]

Xnew, Ynew, id0, id1 = collect_rrps_windows_from_pairs(pairs=pairs)

In [ ]:
def merge_datasets_with_ids(X_old, Y_old,
                            X_new, Y_new,
                            id0_new, id1_new):
    """
    Old data has no id labels -> we pad with -1.
    """
    N0 = X_old.size(0)
    N1 = X_new.size(0)

    X_all = torch.cat([X_old, X_new], dim=0)
    Y_all = torch.cat([Y_old, Y_new], dim=0)

    # id0/id1: -1 for old samples
    id0_old = -torch.ones(N0, dtype=torch.long)
    id1_old = -torch.ones(N0, dtype=torch.long)

    id0_all = torch.cat([id0_old, id0_new], dim=0)
    id1_all = torch.cat([id1_old, id1_new], dim=0)

    return X_all, Y_all, id0_all, id1_all


In [ ]:
Xall, Yall, id0all, id1all = merge_datasets_with_ids(Xc, Yc, Xnew, Ynew, id0, id1)

In [ ]:
def mirror_augment(X, Y, id0, id1):
    """
    X:   (N, L) joint token sequences where each token ∈ {0..8, 9=PAD}
    Y:   (N,)   next-token labels
    id0: (N,)   seat 0 ids
    id1: (N,)   seat 1 ids

    Returns concatenated original + mirrored.
    """
    device = X.device
    PAD = 9

    # mapping for token swap: encode_joint(a0,a1) → encode_joint(a1,a0)
    swap_map = torch.tensor([0, 3, 6, 1, 4, 7, 2, 5, 8, 9], device=device)
    # indices: 0..8 joint tokens, 9 = PAD (maps to itself)

    X_m = swap_map[X]  # swap a0<->a1 in every token
    Y_m = swap_map[Y]

    id0_m = id1.clone()
    id1_m = id0.clone()

    X_all  = torch.cat([X, X_m], dim=0)
    Y_all  = torch.cat([Y, Y_m], dim=0)
    id0_all = torch.cat([id0, id0_m], dim=0)
    id1_all = torch.cat([id1, id1_m], dim=0)

    return X_all, Y_all, id0_all, id1_all


In [ ]:
Xm, Ym, id0m, id1m = mirror_augment(Xall, Yall, id0all, id1all)

In [ ]:
print(Xm.shape, Ym.shape, id0m.shape, id1m.shape)

torch.Size([995004, 100]) torch.Size([995004]) torch.Size([995004]) torch.Size([995004])


# Predictor Model (Transformer+Memory)

In [ ]:
import torch
import torch.nn as nn

class PredictorModel(nn.Module):
    def __init__(self, d_model=64, n_heads=2, n_layers=3,
                 vocab=10, num_bots=43):
        super().__init__()

        self.d_model = d_model
        self.num_bots = num_bots

        # token + position embeddings
        self.token_emb = nn.Embedding(vocab, d_model)
        self.pos_emb = nn.Embedding(200, d_model)

        # transformer encoder
        enc_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=4 * d_model,
            batch_first=True,
        )
        self.transformer = nn.TransformerEncoder(enc_layer, num_layers=n_layers)

        # main head: next joint action (9 classes)
        self.head = nn.Linear(d_model, 9)

        # NEW: identity head – predicts opponent bot id (0..num_bots-1)
        self.id_head = nn.Linear(d_model, num_bots)

    def forward(self, tokens):
        B, L = tokens.shape

        pos = torch.arange(L, device=tokens.device).unsqueeze(0).expand(B, L)
        x = self.token_emb(tokens) + self.pos_emb(pos)

        h = self.transformer(x)
                               # (B, L, d_model)
        hpred = h[:,-1,:]
        h0 = h[:,0,:]
        h1 = h[:,1,:]

        # prediction uses the last time-step token
        pred_logits = self.head(hpred)          # (B, 9)


        # identity prediction uses the first and second time-step token
        id0_logits = self.id_head(h0)          # (B, num_bots)
        id1_logits = self.id_head(h1)

        return pred_logits, id0_logits, id1_logits, hpred, h0, h1


In [ ]:
subset = list(range(20))   # use bots 0–19 (covers deterministic + reactive bots)
trajectories = collect_rrps_data(env, roshambo_bot_names[:20], episodes_per_pair=1)
print(len(trajectories))

400


In [ ]:
pairs = [
    (43, 40),
    (40, 43),
    (43, 20),
    (28,43),
    (43,28),
    (42,40),
    (40,42),
    (42,20),
    (20,42),
    (40,20),
    (20,40),
    (28,40),
    (40,28),
    (28,20),
    (20,28),
    (28,42),
    (42,28),
    (20, 43),
    (43, 42),
]

trajectories2 = collect_pairs_data(env, roshambo_bot_names, pairs, episodes_per_pair=2)


In [ ]:
print(len(trajectories2))

38


In [ ]:
X, Y = make_training_samples(trajectories, window=100)
print(X.shape, Y.shape)


torch.Size([399600, 100]) torch.Size([399600])


In [ ]:
X2, Y2 = make_training_samples(trajectories2+selftrajectory, window=100)

In [ ]:
print(Xc.shape,Yc.shape)

torch.Size([440559, 100]) torch.Size([440559])


In [ ]:
print(X2.shape,Y2.shape)

torch.Size([40959, 100]) torch.Size([40959])


In [ ]:
print(X.shape,Y.shape)

torch.Size([399600, 100]) torch.Size([399600])


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = "/content/drive/MyDrive/rrpsm_data.pt"

In [ ]:
import torch

# Replace the filename below if yours differs
data = torch.load(file_path, map_location="cpu")
Xm = data["Xm"]
Ym = data["Ym"]
id0m = data["id0m"]
id1m = data["id1m"]
window = data.get("window", 100)

print("Loaded:", Xm.shape, Ym.shape, id0m.shape, id1m.shape)
print("Window:", window)


Loaded: torch.Size([995004, 100]) torch.Size([995004]) torch.Size([995004]) torch.Size([995004])
Window: 100


# Symmetry loss

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
def swap_tokens_batch(batch_tokens):
    PAD = 9
    a0 = batch_tokens // 3
    a1 = batch_tokens % 3
    swapped = a1 * 3 + a0
    swapped[batch_tokens == PAD] = PAD
    return swapped

swap_map = torch.tensor([0,3,6,1,4,7,2,5,8], device=device)

def symmetry_loss(pred, pred_sw):
    pred_swapped = pred[:, swap_map]
    return ((pred_sw - pred_swapped)**2).mean()

# Training Loop

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = PredictorModel(num_bots=len(roshambo_bot_names), n_layers=3).to(device)
model2 = PredictorModel(num_bots=len(roshambo_bot_names), n_layers=3).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-4)
opt2 = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_ce  = nn.CrossEntropyLoss()
loss_idc = nn.CrossEntropyLoss()

In [ ]:
for g in opt.param_groups:
    g["lr"] = 1e-3

In [ ]:
beta_id = 0.0   # weight for identity loss
lambda_sym = 0.00
batch_size = 512
num_epochs = 10
save_every = 200          # save every 200 batches
show_every = 200

In [ ]:
Xm_dev  = Xm.to(device)
Ym_dev  = Ym.to(device)
id0m_dev = id0m.to(device)
id1m_dev = id1m.to(device)

NameError: name 'Xm' is not defined

In [ ]:
import os

ckpt_path_latest = "/content/drive/MyDrive/pred_checkpoint_latest.pt"
ckpt_path_latest2 = "/content/drive/MyDrive/pred_checkpoint_latest2.pt"

start_epoch = 0

if os.path.exists(ckpt_path_latest):
    ckpt = torch.load(ckpt_path_latest, map_location=device)
    model.load_state_dict(ckpt["model"])
    opt.load_state_dict(ckpt["opt"])
    start_epoch = ckpt["epoch"]
if os.path.exists(ckpt_path_latest2):
    ckpt = torch.load(ckpt_path_latest2, map_location=device)
    model2.load_state_dict(ckpt["model"])
    opt2.load_state_dict(ckpt["opt"])
    start_epoch = ckpt["epoch"]
else:
    model2.load_state_dict(torch.load("/content/drive/MyDrive/online_pred_weights.pt", map_location=device))

    print(f"Resuming from epoch {start_epoch}, batch {ckpt['idx']}")

In [ ]:
print("Training on:", device)

N = Xm_dev.size(0)


ckpt_path_latest = "/content/drive/MyDrive/pred_checkpoint_latest.pt"
ckpt_path_epoch  = "/content/drive/MyDrive/pred_checkpoint_epoch{}.pt"

for epoch in range(num_epochs):
    perm = torch.randperm(N, device=device)
    running_ce = 0.0

    for idx in range(0, N, batch_size):
        batch_ix = perm[idx:idx + batch_size]
        inp = Xm_dev[batch_ix]
        tgt = Ym_dev[batch_ix]
        id0b = id0m_dev[batch_ix]
        id1b = id1m_dev[batch_ix]

        B = inp.size(0)

        pred, id0_logits, id1_logits, hpred, h0, h1 = model(inp)
        loss_main = loss_ce(pred, tgt)

        # identity loss only on samples with valid labels (>=0)
        mask = (id0b >= 0) & (id1b >= 0)
        if mask.any():
            loss_id0 = loss_idc(id0_logits[mask], id0b[mask])
            loss_id1 = loss_idc(id1_logits[mask], id1b[mask])
            loss_id = 0.5 * (loss_id0 + loss_id1)

            frac = mask.float().mean().item()
            scaled_beta = beta_id / max(frac, 1e-6)
        else:
            loss_id = 0.0

        # symmetry loss every few batches
        if idx % (4 * batch_size) == 0:
            swapped_inp = swap_tokens_batch(inp)
            pred_sw, _, _, _, _, _ = model(swapped_inp)
            loss_sym = symmetry_loss(pred, pred_sw)
        else:
            loss_sym = 0.0

        loss = loss_main + lambda_sym * loss_sym + scaled_beta * loss_id

        if idx % (show_every*batch_size) == 0:
            print(f"[epoch {epoch}, batch {idx/batch_size}] "
                  f"loss_main={loss_main.item():.4f}, "
                  f"loss_sym={loss_sym if isinstance(loss_sym, float) else loss_sym.item():.4f}, "
                  f"loss_id={loss_id if isinstance(loss_id, float) else loss_id.item():.4f}")

        opt.zero_grad()
        loss.backward()
        opt.step()

        running_ce += loss_main.item()

        # 🔥 periodic checkpointing (won't kill performance)
        if idx % (save_every*batch_size) == 0:
            checkpoint = {
                "epoch": epoch,
                "idx": idx,
                "model": model.state_dict(),
                "opt": opt.state_dict(),
                "beta_id": beta_id,
                "lambda_sym": lambda_sym,
                "batch_size": batch_size,
            }
            torch.save(checkpoint, ckpt_path_latest)
            print(f"💾 Saved checkpoint at epoch {epoch}, batch {idx//batch_size}")

    # 🔥 save at end of each epoch
    epoch_ckpt = {
        "epoch": epoch,
        "model": model.state_dict(),
        "opt": opt.state_dict(),
    }
    torch.save(epoch_ckpt, ckpt_path_epoch.format(epoch))
    print(f"💾 Saved epoch {epoch} checkpoint")

    print(f"Epoch {epoch}: CE Loss = {running_ce / max(1, N//batch_size):.4f}")
torch.save(model.state_dict(), "/content/drive/MyDrive/pred_weights.pt")


Training on: cuda
[epoch 0, batch 0.0] loss_main=1.5246, loss_sym=0.6108, loss_id=4.0096
💾 Saved checkpoint at epoch 0, batch 0
[epoch 0, batch 200.0] loss_main=1.5391, loss_sym=0.4341, loss_id=4.0884
💾 Saved checkpoint at epoch 0, batch 200
[epoch 0, batch 400.0] loss_main=1.4916, loss_sym=0.4707, loss_id=4.1628
💾 Saved checkpoint at epoch 0, batch 400
💾 Saved epoch 0 checkpoint
Epoch 0: CE Loss = 1.5049
[epoch 1, batch 0.0] loss_main=1.5155, loss_sym=0.4410, loss_id=3.9379
💾 Saved checkpoint at epoch 1, batch 0
[epoch 1, batch 200.0] loss_main=1.4668, loss_sym=0.4507, loss_id=4.3647
💾 Saved checkpoint at epoch 1, batch 200
[epoch 1, batch 400.0] loss_main=1.5167, loss_sym=0.4407, loss_id=4.1701
💾 Saved checkpoint at epoch 1, batch 400
💾 Saved epoch 1 checkpoint
Epoch 1: CE Loss = 1.4979
[epoch 2, batch 0.0] loss_main=1.5442, loss_sym=0.4245, loss_id=4.1437
💾 Saved checkpoint at epoch 2, batch 0
[epoch 2, batch 200.0] loss_main=1.4970, loss_sym=0.4309, loss_id=4.1200
💾 Saved checkpoin

In [ ]:
print("Training on:", device)

N = Xm_dev.size(0)


ckpt_path_latest2 = "/content/drive/MyDrive/pred_checkpoint_latest2.pt"
ckpt_path_epoch  = "/content/drive/MyDrive/pred_checkpoint_epoch{}.pt"

for epoch in range(num_epochs):
    perm = torch.randperm(N, device=device)
    running_ce = 0.0

    for idx in range(0, N, batch_size):
        batch_ix = perm[idx:idx + batch_size]
        inp = Xm_dev[batch_ix]
        tgt = Ym_dev[batch_ix]
        id0b = id0m_dev[batch_ix]
        id1b = id1m_dev[batch_ix]

        B = inp.size(0)

        pred, id0_logits, id1_logits, hpred, h0, h1 = model2(inp)
        loss_main = loss_ce(pred, tgt)

        # identity loss only on samples with valid labels (>=0)
        mask = (id0b >= 0) & (id1b >= 0)
        if mask.any():
            loss_id0 = loss_idc(id0_logits[mask], id0b[mask])
            loss_id1 = loss_idc(id1_logits[mask], id1b[mask])
            loss_id = 0.5 * (loss_id0 + loss_id1)

            frac = mask.float().mean().item()
            scaled_beta = beta_id / max(frac, 1e-6)
        else:
            loss_id = 0.0

        # symmetry loss every few batches
        if idx % (4 * batch_size) == 0:
            swapped_inp = swap_tokens_batch(inp)
            pred_sw, _, _, _, _, _ = model2(swapped_inp)
            loss_sym = symmetry_loss(pred, pred_sw)
        else:
            loss_sym = 0.0

        loss = loss_main + lambda_sym * loss_sym + scaled_beta * loss_id

        if idx % (show_every*batch_size) == 0:
            print(f"[epoch {epoch}, batch {idx/batch_size}] "
                  f"loss_main={loss_main.item():.4f}, "
                  f"loss_sym={loss_sym if isinstance(loss_sym, float) else loss_sym.item():.4f}, "
                  f"loss_id={loss_id if isinstance(loss_id, float) else loss_id.item():.4f}")

        opt2.zero_grad()
        loss.backward()
        opt2.step()

        running_ce += loss_main.item()

        # 🔥 periodic checkpointing (won't kill performance)
        if idx % (save_every*batch_size) == 0:
            checkpoint = {
                "epoch": epoch,
                "idx": idx,
                "model": model2.state_dict(),
                "opt": opt2.state_dict(),
                "beta_id": beta_id,
                "lambda_sym": lambda_sym,
                "batch_size": batch_size,
            }
            torch.save(checkpoint, ckpt_path_latest2)
            print(f"💾 Saved checkpoint at epoch {epoch}, batch {idx//batch_size}")

    # 🔥 save at end of each epoch
    epoch_ckpt = {
        "epoch": epoch,
        "model": model2.state_dict(),
        "opt": opt2.state_dict(),
    }
    torch.save(epoch_ckpt, ckpt_path_epoch.format(epoch))
    print(f"💾 Saved epoch {epoch} checkpoint")

    print(f"Epoch {epoch}: CE Loss = {running_ce / max(1, N//batch_size):.4f}")
torch.save(model2.state_dict(), "/content/drive/MyDrive/pred_weights.pt")


Training on: cpu
[epoch 0, batch 0.0] loss_main=1.4095, loss_sym=0.5027, loss_id=4.1506
💾 Saved checkpoint at epoch 0, batch 0


KeyboardInterrupt: 

In [ ]:
from google.colab import files
files.download("predictor_weights.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download("predictor_weights2.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Basic Template for an RL agent

Below is a template of a basic RL agent to get you started. You will be implementing the step() function.

In [ ]:
class IdentityBelief():
    def __init__(self, num_bots=43, alpha=0.15, device='cpu'):
        self.alpha = alpha
        self.belief = torch.ones(num_bots).to(device) / num_bots

    def update(self, logits):
        probs = torch.softmax(logits, dim=-1)
        self.belief = (1 - self.alpha) * self.belief + self.alpha * probs
        return self.belief

In [ ]:
class IdentityConditionedPolicy(nn.Module):
    def __init__(self, d_model, num_bots=43):
        super().__init__()

        # each identity dist is one "token" -> shape (2, num_bots)
        self.id_to_Q = nn.Linear(num_bots, d_model)
        self.id_to_K = nn.Linear(num_bots, d_model)
        self.id_to_V = nn.Linear(num_bots, d_model)

        # base latent features: h0, h1, hpred, pred_logits (9-dim)
        self.feature_proj = nn.Linear(d_model*3 + 9, d_model)

        # final action head
        self.out = nn.Linear(d_model, 3)

    def forward(self, h0, h1, hpred, pred_logits, my_belief, opp_belief):
        # identity tokens: shape (2, num_bots)
        tokens = torch.stack([my_belief, opp_belief], dim=0)  # (2, 43)

        Q = self.id_to_Q(tokens)  # (2, d)
        K = self.id_to_K(tokens)  # (2, d)
        V = self.id_to_V(tokens)  # (2, d)

        # attention scores: (2,2)
        scores = (Q @ K.t()) / (Q.size(-1)**0.5)
        attn = torch.softmax(scores, dim=-1)  # (2,2)

        # identity-conditioned context: (2,d) -> pooled -> (1,d)
        W = attn @ V
        context = W.mean(dim=0, keepdim=True)

        # base features
        base = torch.cat([h0, h1, hpred, pred_logits], dim=-1)
        base = self.feature_proj(base.unsqueeze(0))  # (1,d)

        # combine context
        fused = torch.tanh(base + context)  # (1,d)

        # action logits
        return self.out(fused)  # (1,3)


In [ ]:
class IdentityTemperature(nn.Module):
    def __init__(self, num_bots=43):
        super().__init__()
        self.w = nn.Parameter(torch.zeros(num_bots))
        self.a = nn.Parameter(torch.tensor(1.0))   # gain
        self.b = nn.Parameter(torch.tensor(1.0))   # base temp

    def forward(self, id0_logits, id1_logits):
        # identity difference
        d = id0_logits - id1_logits   # shape (43)

        # advantage score
        s = torch.dot(self.w, d)

        # map to temperature
        tau = F.softplus(self.a * s + self.b)

        # clamp tau to reasonable range
        tau = torch.clamp(tau, 0.3, 2.5)

        return tau


In [ ]:
class TemperatureHead(nn.Module):
    def __init__(self, num_bots=43):
        super().__init__()

        self.mlp = nn.Sequential(
            nn.Linear(num_bots, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

    def forward(self, my_belief, opp_belief):
        # difference tells us how much "we are like X" vs "they are like X"
        diff = my_belief - opp_belief      # (num_bots,)
        raw = self.mlp(diff)               # (1,)

        tau = F.softplus(raw)[0]           # positive scalar
        tau = torch.clamp(tau, 0.1, 5.0)   # keep it sane
        return tau


In [ ]:
class DynamicPayoffHead(nn.Module):
    def __init__(self, d_model=64, hidden=128, max_delta=0.5):
        super().__init__()
        self.max_delta = max_delta   # clamp magnitude

        # Combine the three latent vectors -> one fused representation
        self.fuse = nn.Sequential(
            nn.Linear(d_model * 3, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
        )

        # Output 9 numbers (3×3 payoff adjustments)
        self.out = nn.Linear(hidden, 9)

    def forward(self, h0, h1, hpred):
        """
        h0, h1, hpred are each (d_model,)
        Output: ΔR of shape (3,3)
        """

        x = torch.cat([h0, h1, hpred], dim=-1)
        z = self.fuse(x)

        raw = self.out(z)                 # (9,)
        delta = torch.tanh(raw) * self.max_delta
        delta = delta.view(3, 3)          # reshape

        # OPTIONAL: zero the diagonal
        # delta = delta * (1 - torch.eye(3))

        return delta


In [ ]:
class QHead(nn.Module):
    def __init__(self, d_model=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d_model, 64),
            nn.ReLU(),
            nn.Linear(64, 3)   # Q-values for R,P,S
        )

    def forward(self, hpred):
        return self.net(hpred)   # (3,)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


# ============================================================
#      1)   ANALYTICAL BEST-RESPONSE FROM pred_logits
# ============================================================

def compute_expected_reward_from_pred(pred_logits):
    """
    pred_logits: shape (9,)
    Returns: tensor (3,) expected reward for actions R,P,S
    """

    probs = torch.softmax(pred_logits, dim=-1)  # (9,)
    M = probs.view(3, 3)                        # joint matrix

    Rmat = torch.tensor([
        [ 0, -1,  1],
        [ 1,  0, -1],
        [-1,  1,  0],
    ], dtype=torch.float32, device=pred_logits.device)

    ER = torch.zeros(3, device=pred_logits.device)
    for a0 in range(3):
        row = M[a0]
        s = row.sum()
        if s <= 1e-9:
            ER[a0] = 0
        else:
            cond = row / s
            ER[a0] = torch.sum(Rmat[a0] * cond)

    return ER   # shape (3,)


In [ ]:
class PolicyNet(nn.Module):
    def __init__(self, d_model=64, num_bots=43, max_delta=0.5):
        super().__init__()

        self.dynamic_R = DynamicPayoffHead(d_model=d_model, max_delta=max_delta)

        # identity → temperature
        self.w = nn.Parameter(torch.zeros(num_bots))
        self.b = nn.Parameter(torch.tensor(1.0))

        # final fusion
        self.head = nn.Linear(3, 3)

        # static base RPS matrix
        self.R_static = torch.tensor([
            [ 0, -1,  1],
            [ 1,  0, -1],
            [-1,  1,  0]
        ], dtype=torch.float32)

    def compute_temperature(self, my_b, opp_b):
        diff = my_b - opp_b
        tau_raw = (self.w * diff).sum() + self.b
        tau = F.softplus(tau_raw).clamp(0.1, 5.0)
        return tau

    def forward(self, h0, h1, hpred, pred_logits, my_b, opp_b):
        # --- predicted opponent distribution ---
        probs = torch.softmax(pred_logits, dim=-1).view(3,3)

        # --- compute dynamic payoff ---
        delta = self.dynamic_R(h0, h1, hpred)  # (3,3)
        R_eff = self.R_static.to(delta.device) + delta

        expected = torch.matmul(R_eff, probs.sum(dim=1))

        # --- temperature scaling ---
        tau = self.compute_temperature(my_b, opp_b)
        scaled = expected / tau

        # --- little linear head for flexibility ---
        logits = self.head(scaled)

        return logits


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import copy


class MyAgent(rl_agent.AbstractAgent):
    def __init__(self, num_actions, model,
                 weights="pred_weights.pt",
                 d_model=64,
                 num_bots=43, update_pred=False, device="cpu", player_id=0, name='predynamic'):

        self.device = device
        self._num_actions = num_actions
        self.name = name
        self.update_pred = update_pred
        self.num_bots = num_bots
        self.player_id = player_id

        # ---------------- FROZEN PREDICTOR ----------------
        if model is None:
            self.predictor = PredictorModel(
                d_model=d_model, n_layers=3, num_bots=num_bots
            ).to(device)
            self.predictor.load_state_dict(
                torch.load(weights, map_location="cpu"), strict=False
            )
        else:
            self.predictor = copy.deepcopy(model).to(device)

        self.predictor.eval()
        for p in self.predictor.parameters():
            p.requires_grad = update_pred

        # ---------------- POLICY (TRAINED HEADS) ----------------
        self.policy = PolicyNet(d_model=d_model, num_bots=num_bots).to(device)

        # ---------------- BELIEFS + HISTORY ---------------
        self.my_belief = IdentityBelief(num_bots,device=device)
        self.opp_belief = IdentityBelief(num_bots, device=device)
        self.history_tokens = []
        self.saved_last = None


    # ------------ helpers ------------

    def restart(self):
        self.history_tokens = []
        self.my_belief = IdentityBelief(self.num_bots, device=self.device)
        self.opp_belief = IdentityBelief(self.num_bots, device=self.device)
        self.saved_last = None

    def _encode_move(self, a0, a1):
        return 3 * a0 + a1

    def _make_window(self):
        win = self.history_tokens[-100:]
        if len(win) < 100:
            win = [9] * (100 - len(win)) + win
        return torch.tensor(win).long().unsqueeze(0).to(self.device)


    # ------------ main step ------------

    def _step(self, time_step):

        # FIRST STEP
        if time_step.first():
            self.restart()
            probs = np.ones(3) / 3
            action = np.random.choice(3, p=probs)

            self.saved_last = (
                torch.zeros(64), torch.zeros(64), torch.zeros(64),
                torch.zeros(9),
                torch.zeros(self.num_bots),
                torch.zeros(self.num_bots),
                torch.log(torch.tensor(1/3.0)),
            )
            return rl_agent.StepOutput(action=action, probs=probs)

        # DECODE LAST MOVE
        game, state = pyspiel.deserialize_game_and_state(
            time_step.observations["serialized_state"]
        )
        h = state.history()

        if len(h) >= 2:
            a0 = h[-2]
            a1 = h[-1]
        if self.player_id == 0:
            my_a = a0
            opp_a = a1
        else:
            my_a = a1
            opp_a = a0
        self.history_tokens.append(self._encode_move(my_a, opp_a))

        # predictor forward
        window = self._make_window()

        if self.update_pred:
            pred_logits, id0_logits, id1_logits, hpred, h0, h1 = \
                self.predictor(window)
        else:
            with torch.no_grad():
                pred_logits, id0_logits, id1_logits, hpred, h0, h1 = \
                    self.predictor(window)

        # update beliefs
        my_belief = self.my_belief.update(id0_logits[0])
        opp_belief = self.opp_belief.update(id1_logits[0])

        # --- policy forward ---
        logits = self.policy(
            h0[0], h1[0], hpred[0],
            pred_logits[0],
            my_belief,
            opp_belief,
        )

        action_probs = torch.softmax(logits, dim=-1)
        action = torch.multinomial(action_probs, 1).item()
        logp = torch.log(action_probs[action] + 1e-12)

        # store step
        self.saved_last = (
            h0[0],
            h1[0],
            hpred[0],
            pred_logits[0],
            my_belief.clone(),
            opp_belief.clone(),
            logp,
        )

        return rl_agent.StepOutput(
            action=action,
            probs=action_probs.detach().cpu().numpy()
        )
    def step(self, time_step, is_evaluation=True):
        if is_evaluation:
            with torch.no_grad():
                return self._step(time_step)
        else:
            return self._step(time_step)

# RL Training (with REINFORCE)

In [ ]:
def collect_episode(env, agent, opponent):
    trajectory = []

    ts = env.reset()
    agent.restart()
    opponent.restart()

    while not ts.last():
        a_out = agent.step(ts,is_evaluation=False)
        o_out = opponent.step(ts)
        acts = [a_out.action, o_out.action]

        ts_next = env.step(acts)
        reward = ts_next.rewards[0]

        if agent.saved_last is None:
            # safety fallback (should not happen normally)
            h0 = torch.zeros(64)
            h1 = torch.zeros(64)
            hpred = torch.zeros(64)
            pred_logits = torch.zeros(9)
            myb = torch.zeros(agent.num_bots)
            oppb = torch.zeros(agent.num_bots)
            logp = torch.log(torch.tensor(1/3.0))
        else:
            h0, h1, hpred, pred_logits, myb, oppb, logp = agent.saved_last

        step = (h0, h1, hpred, pred_logits, myb, oppb, logp, reward)
        trajectory.append(step)

        ts = ts_next

    return trajectory


In [ ]:
def reinforce_update(agent_policy, optimizer, episode, gamma=0.99):
    # Remove empty steps
    episode = [step for step in episode if len(step) > 0]

    if len(episode) == 0:
        return 0.0  # nothing to learn from

    # discounted rewards
    rewards = [step[-1] for step in episode]
    G = []
    R = 0
    for r in reversed(rewards):
        R = r + gamma * R
        G.insert(0, R)
    returns = torch.tensor(G, dtype=torch.float32)

    # normalize
    adv = (returns - returns.mean()) / (returns.std() + 1e-8)

    loss = 0
    for (step, A) in zip(episode, adv):
        logprob = step[-2]      # second last = log π(a|s)
        loss += -(logprob * A)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()


In [ ]:
import torch.nn.functional as F
import numpy as np
import torch.nn.utils

def reinforce_update_round(policy, optimizer, pred_opt, episodes_round, bot_strength,
                           gamma=0.99, diff_scale_strength=0.4,
                           max_grad_norm=1.0, update_pred = False):
    """
    episodes_round: list of (opponent_id, episode)
      - episode is list of steps: (..., logp, reward)
    bot_strength: numpy array of current strengths
    """
    total_loss_out = 0.0
    any_data = False

    # Get the device of the policy network
    policy_device = next(policy.parameters()).device

    for opponent_id, episode in episodes_round:
        if len(episode) == 0:
            continue

        any_data = True

        # Move returns to the correct device
        rewards = [step[-1] for step in episode]
        G = []
        R = 0
        for r in reversed(rewards):
            R = r + gamma * R
            G.insert(0, R)
        returns = torch.tensor(G, dtype=torch.float32, device=policy_device)

        # advantage normalize per-episode
        adv = (returns - returns.mean()) / (returns.std() + 1e-8)

        # difficulty-based scale (bounded via tanh)
        s = bot_strength[opponent_id]
        scale = 1.0 + diff_scale_strength * np.tanh(s)

        ep_loss = 0.0
        for (step, A) in zip(episode, adv):
            # Move step tensors to the correct device before using them
            h0, h1, hpred, pred_logits, myb, oppb, logp, reward = step

            h0_dev = h0.to(policy_device)
            h1_dev = h1.to(policy_device)
            hpred_dev = hpred.to(policy_device)
            pred_logits_dev = pred_logits.to(policy_device)
            myb_dev = myb.to(policy_device)
            oppb_dev = oppb.to(policy_device)
            logp_dev = logp.to(policy_device)

            logits = policy(h0_dev, h1_dev, hpred_dev, pred_logits_dev, myb_dev, oppb_dev)

            # logprob and adv are already on policy_device
            ep_loss += -(logp_dev * A) * scale / len(episodes_round)

        total_loss_out += ep_loss

    if not any_data:
        return 0.0

    optimizer.zero_grad()
    pred_opt.zero_grad()
    total_loss_out.backward()
    if max_grad_norm is not None:
        torch.nn.utils.clip_grad_norm_(policy.parameters(), max_grad_norm)

    optimizer.step()
    if update_pred:
        pred_opt.step()

    return total_loss_out.item()

In [ ]:
save_path = "/content/drive/MyDrive/temp_weights.pt"
pred_path = "/content/drive/MyDrive/online_pred_weights.pt"
save_every = 1
update_pred = True

In [ ]:
agent = MyAgent(3, model=None, weights=pred_path, update_pred=update_pred, device=device)

In [ ]:
print(agent.device)

cpu


In [ ]:
policy = agent.policy.to(device)
optimizer = torch.optim.Adam(policy.parameters(), lr=1e-4)
pred_opt = torch.optim.Adam(agent.predictor.parameters(), lr=5e-6)

In [ ]:
import os

In [ ]:
policy.load_state_dict(torch.load(save_path,map_location='cpu'))
if os.path.exists(pred_path):
    agent.predictor.load_state_dict(torch.load(pred_path, map_location='cpu'))
else:
    torch.save(agent.predictor.state_dict(), pred_path)


In [ ]:
pure_noise = {"randbot", "pibot", "textbot", "debruijn81"}

hard = {"mixed_strategy", "robertot", "foxtrotbot", "markov5", "markovbails" "russrocker4",
        "shofar", "peterbot", "phasenbott", "sunNervebot", "greenberg", "iocainebot"}

special = {"greenberg", "iocainebot", "markovbails", "phasenbott", "markov5"}

superspec = {"markovbails", "phasenbott"}

def sample_opponent(bot_strength, names):
    strengths = np.array(bot_strength, dtype=np.float32)

    # convert “easier” (very negative) → smaller positive weight
    # and “harder” (less negative / positive) → larger weight
    base = (strengths - strengths.mean())/strengths.std()

    # squash to keep extremes reasonable
    base = np.exp(base)  # gentle compression

    # manual downweight for pure random-ish bots
    for i, name in enumerate(names):
        if name in pure_noise:
            base[i] = 0.0
        if name in hard:
            base[i] * 100.0
        else:
            base[i] *= 0.1
        if name in special:
            base[i] *= 10.0
        if name in superspec:
            base[i] *= 2.0
    probs = base / base.sum()
    return np.random.choice(len(names), p=probs)


In [ ]:
num_bots = 43
if os.path.exists("/content/drive/MyDrive/bot_strength.npy"):
    bot_strength = np.load("/content/drive/MyDrive/bot_strength.npy")
    print("🔄 Loaded saved bot strengths.")
else:
    bot_strength = np.full(43, 0.0, dtype=np.float32)
    print("✨ Initialized new bot strengths.")

🔄 Loaded saved bot strengths.


In [ ]:
rounds = 200
bots_per_round = 5  # number of different bots per round
episodes_per_bot = 1
gamma = 0.977237221
k = 4

In [ ]:
ema_alpha = 0.2
ema_perf = None

In [ ]:
print(sorted(zip(roshambo_bot_names, bot_strength), key=lambda x: x[1], reverse=True))

[('phasenbott', np.float32(-0.38462558)), ('randbot', np.float32(-0.55250835)), ('markovbails', np.float32(-0.8252956)), ('pibot', np.float32(-0.9155607)), ('iocainebot', np.float32(-1.1841633)), ('debruijn81', np.float32(-1.7767735)), ('actr_lag2_decay', np.float32(-3.0306308)), ('markov5', np.float32(-3.954097)), ('textbot', np.float32(-4.3785696)), ('greenberg', np.float32(-4.7941546)), ('russrocker4', np.float32(-6.012001)), ('shofar', np.float32(-9.743508)), ('biopic', np.float32(-9.896416)), ('mod1bot', np.float32(-10.075775)), ('boom', np.float32(-10.432556)), ('flatbot3', np.float32(-12.9398365)), ('adddriftbot2', np.float32(-15.487335)), ('robertot', np.float32(-16.168736)), ('sunNervebot', np.float32(-16.579884)), ('switchalot', np.float32(-17.066345)), ('driftbot', np.float32(-17.140211)), ('inocencio', np.float32(-17.680489)), ('foxtrotbot', np.float32(-18.978935)), ('piedra', np.float32(-18.981367)), ('halbot', np.float32(-19.838905)), ('sweetrock', np.float32(-20.445265))

In [ ]:
print(roshambo_bot_ids)

{'actr_lag2_decay': 0, 'adddriftbot2': 1, 'addshiftbot3': 2, 'antiflatbot': 3, 'antirotnbot': 4, 'biopic': 5, 'boom': 6, 'copybot': 7, 'debruijn81': 8, 'driftbot': 9, 'flatbot3': 10, 'foxtrotbot': 11, 'freqbot2': 12, 'granite': 13, 'greenberg': 14, 'halbot': 15, 'inocencio': 16, 'iocainebot': 17, 'marble': 18, 'markov5': 19, 'markovbails': 20, 'mixed_strategy': 21, 'mod1bot': 22, 'multibot': 23, 'peterbot': 24, 'phasenbott': 25, 'pibot': 26, 'piedra': 27, 'predbot': 28, 'r226bot': 29, 'randbot': 30, 'robertot': 31, 'rockbot': 32, 'rotatebot': 33, 'russrocker4': 34, 'shofar': 35, 'sunCrazybot': 36, 'sunNervebot': 37, 'sweetrock': 38, 'switchalot': 39, 'switchbot': 40, 'textbot': 41, 'zq_move': 42}


In [ ]:
for (i, name, strength) in sorted(zip(roshambo_bot_ids.values(), roshambo_bot_names, bot_strength), key=lambda x: x[2], reverse=True):
    print(f"bot={i:02}, name={name:02} strength={strength:.2f}")

bot=25, name=phasenbott strength=2.42
bot=30, name=randbot strength=-0.55
bot=26, name=pibot strength=-0.92
bot=20, name=markovbails strength=-1.27
bot=08, name=debruijn81 strength=-1.78
bot=19, name=markov5 strength=-3.13
bot=14, name=greenberg strength=-3.24
bot=41, name=textbot strength=-4.38
bot=00, name=actr_lag2_decay strength=-5.44
bot=17, name=iocainebot strength=-5.88
bot=34, name=russrocker4 strength=-6.11
bot=35, name=shofar strength=-9.28
bot=05, name=biopic strength=-10.03
bot=06, name=boom strength=-10.86
bot=22, name=mod1bot strength=-12.89
bot=10, name=flatbot3 strength=-14.33
bot=01, name=adddriftbot2 strength=-16.01
bot=31, name=robertot strength=-16.24
bot=39, name=switchalot strength=-17.65
bot=16, name=inocencio strength=-19.58
bot=09, name=driftbot strength=-19.70
bot=27, name=piedra strength=-20.00
bot=37, name=sunNervebot strength=-20.08
bot=15, name=halbot strength=-20.31
bot=38, name=sweetrock strength=-21.80
bot=11, name=foxtrotbot strength=-24.06
bot=29, nam

In [ ]:
print(np.argmax(bot_strength))

25


In [ ]:
print(np.argmin(bot_strength))

32


In [ ]:
for g in optimizer.param_groups:
            g["lr"] = 1e-5

In [ ]:
for g in pred_opt.param_groups:
      g['lr'] = 1e-7

In [ ]:
for id in range(num_bots):
    if roshambo_bot_names[id] in pure_noise:
        bot_strength[id] -= 3

In [ ]:
for it in range(rounds):

    episodes_round = []
    sampled_bot_ids = []
    prev_bot_strength = []
    # ---- collect episodes vs multiple bots ----
    for _ in range(bots_per_round):
        opponent_id = sample_opponent(bot_strength, roshambo_bot_names)
        sampled_bot_ids.append(opponent_id)
        prev_bot_strength.append(bot_strength[opponent_id])
        opp = create_roshambo_bot_agent(
            1, num_actions, roshambo_bot_names, opponent_id
        )
        ep = collect_episode(env, agent, opp)
        episodes_round.append((opponent_id, ep))

        # update difficulty EMA per-episode
        ep_return = sum(step[-1] for step in ep) # Summing over collected CPU tensors
        bot_strength[opponent_id] = (
            0.99 * bot_strength[opponent_id] + 0.01 * (-ep_return)
        )

    sampled_bot_ids = np.array(sampled_bot_ids, dtype=np.int32)
    prev_bot_strength = np.array(prev_bot_strength, dtype=np.float32)

    # ---- REINFORCE update over the whole round ----
    loss = reinforce_update_round(policy=policy,
        optimizer=optimizer,
        pred_opt=pred_opt,
        episodes_round=episodes_round,
        bot_strength=bot_strength,
        gamma=gamma,
        diff_scale_strength=0.4,
        max_grad_norm=1.0,
        update_pred=update_pred
    )


    # track a simple overall EMA performance metric (unweighted)
    round_returns = [sum(step[-1] for step in ep) for _, ep in episodes_round]
    avg_return = np.mean(round_returns) if round_returns else 0.0

    if ema_perf is None:
        ema_perf = avg_return
    else:
        ema_perf = (1 - ema_alpha) * ema_perf + ema_alpha * avg_return

    # ---- LR schedule ----
    if it == 500:
        for g in optimizer.param_groups:
            g["lr"] = 1e-4
    if it == 1500:
        for g in optimizer.param_groups:
            g["lr"] = 5e-5

    # ---- periodic save ----
    if it % save_every == 0:
        torch.save(policy.state_dict(), save_path)
        # Save the agent's predictor which might still be on CPU
        torch.save(agent.predictor.state_dict(), pred_path)
        torch.save(optimizer.state_dict, save_path.replace(".pt", "_opt.pt"))
        np.save("/content/drive/MyDrive/bot_strength.npy", bot_strength)
        print(f"💾 Saved weights at iter {it}")

    # ---- debug print ----
    # show hardest bot this round
    hardest_id = np.argpartition(bot_strength, -k)[-k:]
    print(
    f"[round {it:04}] "
    f"loss={loss:.3f} "
    f"avgR={avg_return:.2f} EMA_R={ema_perf:.2f} |")
    for id in hardest_id:
        print(f"hardest={id:02}({roshambo_bot_names[id]}), "
        f"strength={bot_strength[id]:.2f}"
        )
    for i, id in enumerate(sampled_bot_ids):
        print(f"sampled={id}, {roshambo_bot_names[id]}, "
              f"prev_strength={prev_bot_strength[i]:.2f}, "
              f"curr_strength={bot_strength[id]:.2f}")


💾 Saved weights at iter 0
[round 0000] loss=-0.224 avgR=-17.00 EMA_R=-17.00 |
hardest=08(debruijn81), strength=-1.78
hardest=26(pibot), strength=-0.92
hardest=30(randbot), strength=-0.55
hardest=25(phasenbott), strength=1.07
sampled=19, markov5, prev_strength=-4.21, curr_strength=-3.77
sampled=25, phasenbott, prev_strength=1.10, curr_strength=1.07
sampled=19, markov5, prev_strength=-4.09, curr_strength=-3.77
sampled=14, greenberg, prev_strength=-2.28, curr_strength=-2.02
sampled=17, iocainebot, prev_strength=-3.90, curr_strength=-3.60
💾 Saved weights at iter 1
[round 0001] loss=8.390 avgR=-6.20 EMA_R=-14.84 |
hardest=08(debruijn81), strength=-1.78
hardest=26(pibot), strength=-0.92
hardest=30(randbot), strength=-0.55
hardest=25(phasenbott), strength=1.22
sampled=25, phasenbott, prev_strength=1.07, curr_strength=1.22
sampled=25, phasenbott, prev_strength=1.16, curr_strength=1.22
sampled=19, markov5, prev_strength=-3.77, curr_strength=-3.72
sampled=14, greenberg, prev_strength=-2.02, curr

KeyboardInterrupt: 

# RL Training with PPO

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class IdentityConditionedValue(nn.Module):
    def __init__(self, d_model=64, num_bots=43):
        super().__init__()

        # mirror the structure of IdentityConditionedPolicy
        self.id_to_query = nn.Linear(num_bots * 2, d_model)
        self.id_to_key   = nn.Linear(num_bots * 2, d_model)
        self.id_to_value = nn.Linear(num_bots * 2, d_model)

        self.feature_proj = nn.Linear(d_model * 3 + 9, d_model)
        self.out = nn.Linear(d_model, 1)

    def forward(self, h0, h1, hpred, pred_logits, my_belief, opp_belief):
        # beliefs: (num_bots,), turn into (1, num_bots*2)
        id_feat = torch.cat([my_belief, opp_belief], dim=-1).unsqueeze(0)

        Q = self.id_to_query(id_feat)
        K = self.id_to_key(id_feat)
        V = self.id_to_value(id_feat)

        attn = (Q @ K.transpose(-2, -1)) / (Q.size(-1)**0.5)
        W = torch.softmax(attn, dim=-1) @ V   # (1, d)

        base = torch.cat([h0, h1, hpred, pred_logits], dim=-1)
        base = self.feature_proj(base.unsqueeze(0))  # (1, d)

        ctx = torch.tanh(base + W)   # (1, d)
        v = self.out(ctx)            # (1, 1)

        # return shape (1,)
        return v.view(1)


In [ ]:
def collect_online_episodes(env, agent, opponent_bot_id, num_episodes=10):
    episodes = []

    opp = create_roshambo_bot_agent(1, num_actions, roshambo_bot_names, opponent_bot_id)

    for ep in range(num_episodes):
        agent.restart()
        opp.restart()

        ts = env.reset()
        episode = []

        while not ts.last():

            out_agent = agent.step(ts)
            out_opp   = opp.step(ts)

            action = out_agent.action
            opp_action = out_opp.action
            old_probs = np.array(out_agent.probs, dtype=np.float32)

            ts_next = env.step([action, opp_action])
            reward = ts_next.rewards[0]

            # --- SKIP until saved_last is populated ---
            if agent.saved_last is None:
                ts = ts_next
                continue

            h0, h1, hpred, pred_logits, my_b, opp_b, last_probs = agent.saved_last

            last_probs = np.array(last_probs, dtype=np.float32)

            episode.append(
                (h0, h1, hpred, pred_logits,
                 my_b, opp_b,
                 action, opp_action,
                 last_probs, reward)
            )

            ts = ts_next

        episodes.append(episode)

    return episodes


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

def online_ppo_update(
        policy,
        value_net,
        optimizer,
        episodes,
        save_path="/content/drive/MyDrive/policy_value_ckpt.pt",
        gamma=0.99,
        clip_eps=0.2,
        aux_weight=0.05,
        entropy_weight=0.005,
        max_kl=0.05,
        save_every=10,
        iteration=0
    ):

    total_loss_out = 0.0

    for ep in episodes:

        # ---- compute discounted return ----
        rewards = [step[-1] for step in ep]
        G = []
        R = 0
        for r in reversed(rewards):
            R = r + gamma * R
            G.insert(0, R)
        G = torch.tensor(G, dtype=torch.float32)

        # ---- value network baseline ----
        values = []
        for step in ep:
            h0, h1, hpred, pred_logits, my_b, opp_b, *_ = step
            v = value_net(h0, h1, hpred, pred_logits, my_b, opp_b)

            # ensure shape [1]
            if v.dim() == 0:
                  v = v.unsqueeze(0)

            values.append(v)

        values = torch.cat(values).squeeze()

        # advantages
        A = (G - values.detach())
        A = (A - A.mean()) / (A.std() + 1e-8)
        A = A / 2.0   # extra stabilization

        # ---- PPO update ----
        for (step, A_step, R_target) in zip(ep, A, G):

            R_target = torch.clamp(R_target, -50, 50)


            (h0, h1, hpred, pred_logits, my_b, opp_b,
             action, opp_action, old_probs, reward) = step

            logits = policy(h0, h1, hpred, pred_logits, my_b, opp_b)
            logits = torch.clamp(logits, -10, 10)  # logit clamp

            logprobs = F.log_softmax(logits, dim=-1)[0]
            logp = logprobs[action]
            old_logp = torch.log(torch.tensor(old_probs[action]) + 1e-12)

            # PPO ratio
            ratio = torch.exp(logp - old_logp)
            surr1 = ratio * A_step
            surr2 = torch.clamp(ratio, 1-clip_eps, 1+clip_eps) * A_step
            ppo_loss = -torch.min(surr1, surr2)

            # Value loss
            value_pred = value_net(h0, h1, hpred, pred_logits, my_b, opp_b)
            value_loss = 0.5 * (value_pred.squeeze() - R_target)**2
            value_loss = 0.01 * value_loss


            # Best/worst auxiliary loss
            best_action = (opp_action + 1) % 3
            worst_action = (best_action + 1) % 3

            best_loss  = -logprobs[best_action]
            worst_loss = +logprobs[worst_action]
            aux_loss = best_loss + 0.25 * worst_loss

            # Entropy bonus
            entropy = -(logprobs * torch.exp(logprobs)).sum()
            entropy_loss = -entropy_weight * entropy

            # Full loss
            loss = ppo_loss + value_loss + aux_weight * aux_loss + entropy_loss
            total_loss_out += loss

    # ---- apply gradients ----
    optimizer.zero_grad()
    total_loss_out.backward()
    torch.nn.utils.clip_grad_norm_(policy.parameters(), 1.0)
    torch.nn.utils.clip_grad_norm_(value_net.parameters(), 1.0)
    optimizer.step()

    # ---- Save both networks ----
    if iteration % save_every == 0:
        checkpoint = {
            "policy": policy.state_dict(),
            "value_net": value_net.state_dict(),
            "optimizer": optimizer.state_dict(),
            "iteration": iteration
        }
        torch.save(checkpoint, save_path)
        print(f"💾 Saved policy+value checkpoint at iter {iteration}")

    return total_loss_out.item()


In [ ]:
import numpy as np

def choose_opponent(perf_estimates,
                    temperature=1.0,
                    floor=0.02,           # minimum probability for ANY bot
                    mix_uniform=0.20):    # % of uniform randomness
    """
    perf_estimates: array of running performance estimates vs each bot

    Returns a bot index, avoiding collapse to a single opponent.
    """

    perf = np.array(perf_estimates, dtype=np.float64)

    # difficulty = hardness = lower perf = harder bots
    difficulty = -perf

    # temperature scaling (smooths out distribution)
    scaled = difficulty / max(1e-6, temperature)

    # safe exponentiation
    scaled = np.clip(scaled, -50, 50)   # avoid overflow
    weights = np.exp(scaled)

    # normalize safely
    probs = weights / (weights.sum() + 1e-12)

    # apply probability floor so no bot hits 0
    probs = np.maximum(probs, floor)
    probs = probs / probs.sum()  # renormalize

    # mix in some uniform randomness (encourages exploration)
    if mix_uniform > 0:
        uniform = np.ones_like(probs) / len(probs)
        probs = (1 - mix_uniform) * probs + mix_uniform * uniform

    # final renormalization
    probs = probs / probs.sum()

    # safe choice
    return np.random.choice(len(probs), p=probs)


In [ ]:
perf = np.zeros(43)  # running avg return vs each bot
counts = np.zeros(43) + 1e-6


In [ ]:
agent = MyAgent(3, model=model, weights="pred_weights.pt")

In [ ]:
# ---- initialize policy + value ----
policy = agent.policy
value_net = IdentityConditionedValue(d_model=64, num_bots=43)

In [ ]:
optimizer = torch.optim.Adam(
    list(policy.parameters()) + list(value_net.parameters()),
    lr=1e-4
)

In [ ]:
ckpt = torch.load("policy_value_ckpt.pt")
policy.load_state_dict(ckpt["policy"])
value_net.load_state_dict(ckpt["value_net"])
optimizer.load_state_dict(ckpt["optimizer"])
start_iteration = ckpt["iteration"]


In [ ]:
# curriculum performance stats
perf = np.zeros(43, dtype=np.float32)
counts = np.zeros(43, dtype=np.float32) + 1e-6

train_loss_hist = []
avg_return_hist = []

for it in range(2000):

    # choose opponent using curriculum
    bot_id = choose_opponent(perf, temperature=1.0)

    # collect several episodes
    episodes = collect_online_episodes(env, agent, bot_id, num_episodes=5)

    # compute average return achieved
    ep_returns = [sum(step[-1] for step in ep) for ep in episodes]
    avg_return = float(np.mean(ep_returns))

    # PPO update
    loss = online_ppo_update(
        policy,
        value_net,
        optimizer,
        episodes,
        iteration=it
    )

    # update curriculum stats
    counts[bot_id] += 1
    perf[bot_id] += (avg_return - perf[bot_id]) / counts[bot_id]

    # track learning curves
    train_loss_hist.append(loss)
    avg_return_hist.append(avg_return)

    print(f"[RL] iter={it:04d} "
          f"bot={bot_id:02d} {roshambo_bot_names[bot_id]:18s} "
          f"loss={loss:.3f} return={avg_return:.3f}")


💾 Saved policy+value checkpoint at iter 0
[RL] iter=0000 bot=09 driftbot           loss=1390.335 return=7.600
[RL] iter=0001 bot=27 piedra             loss=3604.765 return=-104.800
[RL] iter=0002 bot=27 piedra             loss=3742.040 return=-112.800
[RL] iter=0003 bot=27 piedra             loss=1985.492 return=-59.600
[RL] iter=0004 bot=27 piedra             loss=2403.616 return=-70.600
[RL] iter=0005 bot=40 switchbot          loss=2206.570 return=61.400
[RL] iter=0006 bot=40 switchbot          loss=1507.684 return=55.200
[RL] iter=0007 bot=27 piedra             loss=2768.951 return=-93.200
[RL] iter=0008 bot=32 rockbot            loss=8381.351 return=192.000
[RL] iter=0009 bot=04 antirotnbot        loss=3245.907 return=-108.000
💾 Saved policy+value checkpoint at iter 10
[RL] iter=0010 bot=07 copybot            loss=47294.137 return=701.800
[RL] iter=0011 bot=04 antirotnbot        loss=3285.820 return=-99.400
[RL] iter=0012 bot=38 sweetrock          loss=2818.983 return=-89.000
[RL] 

KeyboardInterrupt: 

# Offline RL with PPO

In [ ]:
def build_offline_rl_dataset(predictor, X, Y, id0, id1, num_bots=43):
    offline_data = []

    # create belief trackers for offline pass
    my_belief = IdentityBelief(num_bots)
    opp_belief = IdentityBelief(num_bots)

    R = torch.tensor([
        [ 0, -1,  1],   # R vs R,P,S
        [ 1,  0, -1],
        [-1,  1,  0],
    ], dtype=torch.float32)

    with torch.no_grad():
        for i in range(len(X)):
            window = X[i].unsqueeze(0)
            pred_logits, id0_logits, id1_logits, hpred, h0, h1 = predictor(window)

            # update identity EMA
            my_belief.update(id0_logits[0])
            opp_belief.update(id1_logits[0])

            # decode the true action pair from label Y
            joint = Y[i].item()
            a0 = joint // 3
            a1 = joint % 3

            reward = R[a0, a1].item()

            offline_data.append((
                h0[0], h1[0], hpred[0],
                pred_logits[0],
                my_belief.belief.clone(),
                opp_belief.belief.clone(),
                a0,                        # true action
                (a1+1) % 3,                 # best action, reward = 1
                reward                     # true reward
            ))

    return offline_data


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

def offline_ppo_update(
        policy,
        data,
        optimizer,
        epochs=3,
        aux_weight=0.05,
        entropy_weight=0.01,
        save_path="/content/drive/MyDrive/policy_weights_offline.pt",
        save_every=1,
        iteration=0
    ):
    """
    data: list of tuples
      (h0, h1, hpred, pred_logits, my_b, opp_b,
       action, best_action, reward)

    No old_probs stored → treat as advantage regression.
    """

    total_loss_out = 0.0

    for ep in range(epochs):

        total_loss = 0.0

        # Compute reward normalization (advantage normalization)
        rewards = torch.tensor([d[-1] for d in data], dtype=torch.float32)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-8)

        for i, (h0, h1, hpred, pred_logits,
                my_b, opp_b,
                action, best_action, reward) in enumerate(data):

            logits = policy(h0, h1, hpred, pred_logits, my_b, opp_b)
            logprobs = F.log_softmax(logits, -1)[0]

            # --- Reinforcement (advantage-weighted regression) ---
            A = rewards[i]
            reinforce_loss = -logprobs[action] * A

            # --- Auxiliary best-move loss ---
            # best action comes from dataset (precomputed)
            best_loss = -logprobs[best_action]

            # correct worst action (loses to best)
            worst_action = (best_action + 1) % 3
            worst_loss = +logprobs[worst_action]

            aux_loss = best_loss + 0.25 * worst_loss

            # --- Entropy bonus ---
            entropy = -(logprobs * torch.exp(logprobs)).sum()
            entropy_loss = -entropy_weight * entropy

            # --- Combine ---
            loss = reinforce_loss + aux_weight * aux_loss + entropy_loss
            total_loss += loss

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        total_loss_out += float(total_loss.item())

        # Save weights
        if iteration % save_every == 0 and ep == epochs - 1:
            torch.save(policy.state_dict(), save_path)
            print(f"💾 Saved offline policy to {save_path} at iter {iteration}")

    return total_loss_out


In [ ]:
offline_data = build_offline_rl_dataset(agent.predictor, Xm_dev, Ym_dev, id0m_dev, id1m_dev)

KeyboardInterrupt: 

In [ ]:
policy = agent.policy
optimizer = torch.optim.Adam(policy.parameters(), lr=3e-4)

for it in range(50):
    loss = offline_ppo_update(policy, offline_data, optimizer)
    print(f"[offline RL] iter {it}, loss={loss:.3f}")

# Template Agent

In [ ]:
# Template to get you started. Pick one of these to implement as your starting
# point.

# Template : Basic RL agent.
#
#
class MyAgent(rl_agent.AbstractAgent):
  """Agent class that learns to play RRPS.

  You fill this in to create your RRPS agent.

  See the superclass for more info: https://github.com/google-deepmind/open_spiel/blob/master/open_spiel/python/rl_agent.py
  """

  def __init__(self, num_actions, name="bot_agent"):
    assert num_actions > 0
    self._num_actions = num_actions  # 3

  def step(self, time_step, is_evaluation=False):
    # If it is the end of the episode, don't select an action.
    if time_step.last():
      return
    # Note: If the environment was created with include_full_state=True, then
    # game and state can be obtained as follows:

    game, state = pyspiel.deserialize_game_and_state(time_step.observations["serialized_state"])

    # A useful piece of information is the history (previous actions taken by agents).
    # You can access this by state.history()

    # Do something here that selects an action and computes a policy
    # distribution in probs.
    if len(state.history())==0:
      action = 0
    else:
      action = state.history()[-1]
    probs = np.ones(self._num_actions) / self._num_actions
    return rl_agent.StepOutput(action=action, probs=probs)



#A short example to show how to evaluate your agent againt an RRPS bot.

In [ ]:
def evaluate_across_population(env, agent, num_eps=5):
    scores = []
    for bot_id in range(43):
        _, avg_ret, _ = collect_online_episodes(env, agent, bot_id, num_eps)
        scores.append(avg_ret)
    return scores


In [ ]:
# Just trying an example out.

my_agent = MyAgent(3, name="trans_agent")
# print(my_agent._num_actions)


p1_pop_id = 17   # adddriftbot2
agents = [
    my_agent,
    create_roshambo_bot_agent(1, num_actions, roshambo_bot_names, p1_pop_id)
]


print("Starting eval run.")
avg_eval_returns = eval_agents(env, agents, num_players, 4, verbose=True)

print("Avg return ", avg_eval_returns)

RuntimeError: Error(s) in loading state_dict for PredictorModel:
	Missing key(s) in state_dict: "id_head.weight", "id_head.bias". 

In [ ]:
myagent = MyAgent(3, name="trans", weights = "predictor_weights.pt")
myagent2 = MyAgent(3, name="trans2", weights = "predictor_weights2.pt")

In [ ]:
avg_eval_self = eval_agents(env, [my_agent,myagent2], num_players, 8, verbose=True)
print("Avg return", avg_eval_self)

Finished episode 0, avg returns: [ 52. -52.]
Finished episode 1, avg returns: [ 22.5 -22.5]
Finished episode 2, avg returns: [ 20.66666667 -20.66666667]
Finished episode 3, avg returns: [ 6.75 -6.75]
Finished episode 4, avg returns: [ 7. -7.]
Finished episode 5, avg returns: [-2.33333333  2.33333333]
Finished episode 6, avg returns: [-1.  1.]
Finished episode 7, avg returns: [-3.125  3.125]
Avg return [-3.125  3.125]


In [ ]:
# In-sample bot IDs (0–19)
oos_bot_ids = [x for x,_ in sorted(zip(roshambo_bot_ids.values(), bot_strength), key=lambda x: x[1], reverse=True)]

results_oos = {}

for bot_id in oos_bot_ids:
    bot_name = roshambo_bot_names[bot_id]
    opp = create_roshambo_bot_agent(1, num_actions, roshambo_bot_names, bot_id)

    agents = [agent, opp]

    print(f"\n=== Evaluating vs IN-SAMPLE bot {bot_id}: {bot_name} ===")
    avg = eval_agents(env, agents, num_players, 10, verbose=True)
    results_oos[bot_name] = avg[0]

# Print sorted results
print("\n=== In-Sample Performance Ranking (high → low) ===")
for name, score in sorted(results_oos.items(), key=lambda x: x[1], reverse=True):
    print(f"{name:20s} score={score:.2f}")



=== Evaluating vs IN-SAMPLE bot 25: phasenbott ===
Finished episode 0, avg returns: [ 30. -30.]
Finished episode 1, avg returns: [ 7. -7.]
Finished episode 2, avg returns: [-6.66666667  6.66666667]
Finished episode 3, avg returns: [-3.  3.]
Finished episode 4, avg returns: [-0.4  0.4]
Finished episode 5, avg returns: [ 2.16666667 -2.16666667]
Finished episode 6, avg returns: [ 4.57142857 -4.57142857]
Finished episode 7, avg returns: [ 6.625 -6.625]
Finished episode 8, avg returns: [ 3.22222222 -3.22222222]
Finished episode 9, avg returns: [ 0.4 -0.4]

=== Evaluating vs IN-SAMPLE bot 30: randbot ===
Finished episode 0, avg returns: [-4.  4.]
Finished episode 1, avg returns: [-2.5  2.5]
Finished episode 2, avg returns: [-4.66666667  4.66666667]
Finished episode 3, avg returns: [-0.25  0.25]
Finished episode 4, avg returns: [ 6.2 -6.2]
Finished episode 5, avg returns: [-0.16666667  0.16666667]
Finished episode 6, avg returns: [ 1.71428571 -1.71428571]
Finished episode 7, avg returns: [ 2

In [ ]:
# Flipped order
agent.player_id = 1
oos_bot_ids = list(range(0, 43))

results_oos = {}

for bot_id in oos_bot_ids:
    bot_name = roshambo_bot_names[bot_id]
    opp = create_roshambo_bot_agent(1, num_actions, roshambo_bot_names, bot_id)

    agents = [opp, agent]

    print(f"\n=== Evaluating vs IN-SAMPLE bot {bot_id}: {bot_name} ===")
    avg = eval_agents(env, agents, num_players, 10, verbose=True)
    results_oos[bot_name] = avg[1]

# Print sorted results
print("\n=== In-Sample Performance Ranking (high → low) ===")
for name, score in sorted(results_oos.items(), key=lambda x: x[1], reverse=True):
    print(f"{name:20s} score={score:.2f}")



=== Evaluating vs IN-SAMPLE bot 0: actr_lag2_decay ===
Finished episode 0, avg returns: [-18.  18.]
Finished episode 1, avg returns: [-25.5  25.5]
Finished episode 2, avg returns: [-17.33333333  17.33333333]
Finished episode 3, avg returns: [-18.25  18.25]
Finished episode 4, avg returns: [-15.4  15.4]
Finished episode 5, avg returns: [-11.83333333  11.83333333]
Finished episode 6, avg returns: [-10.28571429  10.28571429]
Finished episode 7, avg returns: [-16.75  16.75]
Finished episode 8, avg returns: [-20.11111111  20.11111111]
Finished episode 9, avg returns: [-18.9  18.9]

=== Evaluating vs IN-SAMPLE bot 1: adddriftbot2 ===
Finished episode 0, avg returns: [-33.  33.]
Finished episode 1, avg returns: [-27.  27.]
Finished episode 2, avg returns: [-11.66666667  11.66666667]
Finished episode 3, avg returns: [-10.  10.]
Finished episode 4, avg returns: [-15.4  15.4]
Finished episode 5, avg returns: [-16.83333333  16.83333333]
Finished episode 6, avg returns: [-15.14285714  15.14285714

KeyboardInterrupt: 

In [ ]:
# Out-of-sample bot IDs (20–42)
oos_bot_ids = list(range(20, 43))

results_oos = {}

for bot_id in oos_bot_ids:
    bot_name = roshambo_bot_names[bot_id]
    opp = create_roshambo_bot_agent(1, num_actions, roshambo_bot_names, bot_id)

    agents = [agent, opp]

    print(f"\n=== Evaluating vs OUT-OF-SAMPLE bot {bot_id}: {bot_name} ===")
    avg = eval_agents(env, agents, num_players, 4, verbose=False)
    results_oos[bot_name] = avg[0]

# Print sorted results
print("\n=== Out of Sample Performance Ranking (high → low) ===")
for name, score in sorted(results_oos.items(), key=lambda x: x[1], reverse=True):
    print(f"{name:20s} score={score:.2f}")



=== Evaluating vs OUT-OF-SAMPLE bot 20: markovbails ===

=== Evaluating vs OUT-OF-SAMPLE bot 21: mixed_strategy ===

=== Evaluating vs OUT-OF-SAMPLE bot 22: mod1bot ===

=== Evaluating vs OUT-OF-SAMPLE bot 23: multibot ===

=== Evaluating vs OUT-OF-SAMPLE bot 24: peterbot ===

=== Evaluating vs OUT-OF-SAMPLE bot 25: phasenbott ===

=== Evaluating vs OUT-OF-SAMPLE bot 26: pibot ===

=== Evaluating vs OUT-OF-SAMPLE bot 27: piedra ===

=== Evaluating vs OUT-OF-SAMPLE bot 28: predbot ===

=== Evaluating vs OUT-OF-SAMPLE bot 29: r226bot ===

=== Evaluating vs OUT-OF-SAMPLE bot 30: randbot ===

=== Evaluating vs OUT-OF-SAMPLE bot 31: robertot ===

=== Evaluating vs OUT-OF-SAMPLE bot 32: rockbot ===

=== Evaluating vs OUT-OF-SAMPLE bot 33: rotatebot ===

=== Evaluating vs OUT-OF-SAMPLE bot 34: russrocker4 ===

=== Evaluating vs OUT-OF-SAMPLE bot 35: shofar ===

=== Evaluating vs OUT-OF-SAMPLE bot 36: sunCrazybot ===

=== Evaluating vs OUT-OF-SAMPLE bot 37: sunNervebot ===

=== Evaluating vs 

# Performance Tracking

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_heatmap(history):
    # history: list of lists (scores vs 43 bots)
    arr = np.array(history)
    plt.figure(figsize=(10,7))
    plt.imshow(arr.T, aspect='auto', cmap='coolwarm', vmin=-1, vmax=1)
    plt.colorbar()
    plt.xlabel("Evaluation iteration")
    plt.ylabel("Bot ID")
    plt.title("Performance vs Bot Population Over Time")
    plt.show()


In [ ]:
def update_elo(my_elo, opp_elo, result, k=32):
    # result: win=1, draw=0.5, lose=0
    expect = 1 / (1 + 10 ** ((opp_elo - my_elo) / 400))
    return my_elo + k * (result - expect)


In [ ]:
def entropy(p):
    return -(p * torch.log(p + 1e-12)).sum().item()

id_entropies = []

# Every episode:
id_entropies.append(entropy(agent.opp_belief.belief))



In [ ]:
plt.plot(id_entropies)
plt.title("Opponent Identity Entropy Over Time")
plt.xlabel("Episode")
plt.ylabel("Entropy")
plt.show()


In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

class RPSPerformanceMonitor:
    def __init__(self, env, agent, bot_names, num_bots=43, num_actions=3):
        self.env = env
        self.agent = agent
        self.bot_names = bot_names
        self.num_bots = num_bots
        self.num_actions = num_actions

        # -----------------------------------------
        # Fixed-size buffers
        # -----------------------------------------
        self.max_latent      = 5000
        self.max_entropy     = 2000
        self.max_cycles      = 2000
        self.max_population  = 200
        self.max_elo         = 2000

        # tracking storage
        self.performance_log = []          # small already
        self.population_history = []       # (eval snapshots)
        self.id_entropy_log = []
        self.cycle_log = []
        self.elo_log = []
        self.latent_log = []

        # ELO scores
        self.agent_elo = 1500
        self.bot_elos = [1500]*num_bots

    # -----------------------------------------
    # Helper: ring buffer append
    # -----------------------------------------
    def _append_limited(self, arr, x, max_len):
        arr.append(x)
        if len(arr) > max_len:
            arr.pop(0)

    # -----------------------------------------
    # Utility functions
    # -----------------------------------------
    def _entropy(self, p):
        return -(p * torch.log(p + 1e-12)).sum().item()

    def _update_elo(self, my_elo, opp_elo, result, k=32):
        expected = 1 / (1 + 10 ** ((opp_elo - my_elo) / 400))
        return my_elo + k * (result - expected)

    def _cycle_strength(self, actions):
        if len(actions) < 2:
            return 0
        score = 0
        for i in range(1, len(actions)):
            prev = actions[i-1]; cur = actions[i]
            if (prev + 1) % 3 == cur:
                score += 1
            elif (prev + 2) % 3 == cur:
                score -= 1
        return score / len(actions)

    # -----------------------------------------
    # Episode collection + evaluations
    # -----------------------------------------
    def collect_and_log(self, iteration, bot_id, num_eps=5):
        opp = create_roshambo_bot_agent(
            1, self.num_actions, self.bot_names, bot_id
        )

        returns = []
        latent_batch = []
        actions_batch = []

        for ep in range(num_eps):
            self.agent.restart()
            opp.restart()
            ts = self.env.reset()
            ep_return = 0

            while not ts.last():

                outs = [self.agent.step(ts), opp.step(ts)]
                actions = [outs[0].action, outs[1].action]
                ts_next = self.env.step(actions)
                reward = ts_next.rewards[0]
                ep_return += reward

                # saved values from MyAgent.forward()
                h0, h1, hpred, pred_logits, my_b, opp_b, last_probs = self.agent.saved_last
                latent_batch.append(torch.cat([h0, h1, hpred]).cpu().numpy())
                actions_batch.append(actions[0])

                ts = ts_next

            returns.append(ep_return)

        avg_return = sum(returns) / len(returns)

        # identity entropy
        ent = self._entropy(self.agent.opp_belief.belief)
        self._append_limited(self.id_entropy_log, ent, self.max_entropy)

        # cycle score
        cycle_s = self._cycle_strength(actions_batch)
        self._append_limited(self.cycle_log, cycle_s, self.max_cycles)

        # latent vectors
        for v in latent_batch:
            self._append_limited(self.latent_log, v, self.max_latent)

        return avg_return

    # -----------------------------------------
    # After PPO update
    # -----------------------------------------
    def after_update(self, iteration, bot_id, avg_return, loss):
        self.performance_log.append((iteration, bot_id, avg_return, loss))

        # ELO update
        if avg_return > 0:
            result = 1
        elif avg_return < 0:
            result = 0
        else:
            result = 0.5

        self.agent_elo = self._update_elo(self.agent_elo, self.bot_elos[bot_id], result)
        self.bot_elos[bot_id] = self._update_elo(self.bot_elos[bot_id], self.agent_elo, 1-result)

        self._append_limited(self.elo_log, self.agent_elo, self.max_elo)

    # -----------------------------------------
    # Population eval
    # -----------------------------------------
    def evaluate_population(self, num_eps=3):
        scores = []
        for b in range(self.num_bots):
            avg_ret = self.collect_and_log(0, b, num_eps)
            scores.append(avg_ret)
        self._append_limited(self.population_history, scores, self.max_population)
        return scores

    # -----------------------------------------
    # Plots
    # -----------------------------------------
    def plot_heatmap(self):
        if not self.population_history:
            print("No population eval yet.")
            return
        arr = np.array(self.population_history)
        plt.figure(figsize=(10,6))
        plt.imshow(arr.T, aspect='auto', cmap='coolwarm', vmin=-1, vmax=1)
        plt.colorbar()
        plt.title("Performance vs All Bots Over Time")
        plt.xlabel("Evaluation Index")
        plt.ylabel("Bot ID")
        plt.show()

    def plot_elo(self):
        if not self.elo_log:
            print("No ELO data yet.")
            return
        plt.plot(self.elo_log)
        plt.title("Agent ELO Over Time")
        plt.xlabel("Iteration")
        plt.ylabel("ELO")
        plt.show()

    def plot_identity_entropy(self):
        if not self.id_entropy_log:
            print("No entropy data yet.")
            return
        plt.plot(self.id_entropy_log)
        plt.title("Opponent Identity Entropy")
        plt.xlabel("Episode")
        plt.ylabel("Entropy")
        plt.show()

    def plot_cycle_strength(self):
        if not self.cycle_log:
            print("No cycle data yet.")
            return
        plt.plot(self.cycle_log)
        plt.title("Cycle Strength Over Time")
        plt.xlabel("Iteration")
        plt.ylabel("Cycle Score")
        plt.show()

    def plot_latent_pca(self, sample=3000):
        if not self.latent_log:
            print("No latent vectors yet.")
            return
        X = np.array(self.latent_log[-sample:])
        if len(X) < 2:
            print("Not enough latent data for PCA.")
            return
        p = PCA(2).fit_transform(X)
        plt.scatter(p[:,0], p[:,1], s=3)
        plt.title("Latent Drift (PCA of h0,h1,hpred)")
        plt.show()


In [ ]:
evaluator = RPSPerformanceMonitor(env, agent, roshambo_bot_names)

for it in range(200):
    bot_id = it % 43

    episodes, avg_return = evaluator.collect_and_log(
        iteration=it,
        bot_id=bot_id,
        num_eps=5
    )

    loss = online_ppo_update(agent.policy, optimizer, episodes)

    evaluator.after_update(it, bot_id, avg_return, loss)

    print(f"[{it}] vs {bot_id}: return={avg_return:.3f} loss={loss:.3f}")

    # periodic wide eval
    if it % 20 == 0:
        print("Population eval...")
        scores = evaluator.evaluate_population(num_eps=3)


In [ ]:
evaluator.plot_elo()
evaluator.plot_heatmap()
evaluator.plot_identity_entropy()
evaluator.plot_cycle_strength()
evaluator.plot_latent_pca()
